# Estructura de anotaciones en CANTEMIST

En cantemist-norm los datos se presentan de la siguiente manera:
- nota1.txt
- nota1.ann <- Fichero de anotación

La anotación viene en un fichero de texto por filas. Cada dos filas están organizadas de la siguiente forma:
La primera fila contiene esta información (sin header)

| indice término | categoria | start char offset | end char offset | mention string |
|----|-----|-----|-----|----|
| T1 | MORFOLOGIA_NEOPLASIA | 3332 | 3341 | Carcinoma miocrítico |

La segunda fila tiene esta información (sin header)
| indice | Annotator Notes | indice término | eCIE-O 3.1 code |
|----|----|----|----|
| #1| AnnotatorNotes | T1 | 8041/3 |

La anotación en la segunda fila se corresponde al término descrito en la primera. Para parsear estas anotaciones a un formato más legible he implementado la siguiente celda con la función `parse_cantemist_annotation()`.


In [16]:
import os
import pandas as pd


def parse_cantemist_annotation(ann_path):
    ''' Parsea anotaciones de Cantemist para hacer un dataframe estructurado.
    
        Input: Un fichero de anotación de cantemist. 
        Output: Un dataframe con la anotación estructurada.
    '''
    # Comprueba que el fichero no esté vacío
    if os.path.getsize(ann_path) == 0:
        return pd.DataFrame(columns=['term_idx', 'text', 'category', 'char_start', 'char_end', 'ICD-O Code'])
    
    # Lee el fichero de anotación
    ann = pd.read_csv(
        ann_path,
        sep = '\t',
        dtype=str,
        encoding='utf-8',
        header=None
    )

    # Parsea la parte de las coordenadas
    nota_coords = ann[ann[0].str.startswith('T')].copy()
    split = nota_coords[1].str.split(' ', expand=True)
    nota_coords = pd.concat([nota_coords[[0, 2]], split], axis=1)
    nota_coords.columns = ['term_idx', 'text', 'category', 'char_start', 'char_end']

    # Parsea las filas de anotación
    nota_ann = ann[ann[0].str.startswith('#')].copy()
    nota_ann[0] = nota_ann[0].str.replace('#', 'T')
    nota_ann = nota_ann.drop(1, axis=1)
    nota_ann.columns = ['term_idx', 'ICD-O Code']

    # Merge de las dos partes
    merged = pd.merge(nota_coords, nota_ann, on='term_idx', how='inner')

    return merged


In [2]:
data_path = "../.data/Cantemist/dev-set1/cantemist-norm/"
nota = "cc_onco853.ann"

df = parse_cantemist_annotation(data_path + nota)

print(df.head())


Empty DataFrame
Columns: [term_idx, text, category, char_start, char_end, ICD-O Code]
Index: []


# Frecuencia de términos
Ahora que sé parsear las anotaciones de Cantemist, quiero ver cuál es el término más frecuente en las notas. Para ello voy a leer todas las notas de .data/Cantemist/dev-set1/cantemist-norm/ e ir anotación por anotación contando los códigos ICD-O que aparecen.

In [3]:
import os


data_path = '../.data/Cantemist/dev-set1/cantemist-norm/'

dict_counts = {}

for nota in os.listdir(data_path):
    if nota.endswith('.ann'):
        df = parse_cantemist_annotation(data_path + nota)
        
        # Cuenta el número de veces que aparece cada código ICD-O
        code_counts = df['ICD-O Code'].value_counts()
        for code, count in code_counts.items():
            if code in dict_counts:
                dict_counts[code] += count
            else:
                dict_counts[code] = count

In [4]:
# Convierte el diccionario a un dataframe para visualizarlo mejor
counts_df = pd.DataFrame(list(dict_counts.items()), columns=['ICD-O Code', 'Count'])
counts_df = counts_df.sort_values(by='Count', ascending=False)

# Carga el mapa de códigos a términos
ecie_umls_map = pd.read_csv(
    "../.data/mappings/ICD-O-3.1-NCIt_Morphology_Mapping.txt",
    sep = '\t',
    dtype=str,
    encoding='utf-8'
)

# Añade a cada código su término preferido
counts_df = counts_df.merge(
    ecie_umls_map[ecie_umls_map['Term Type'] == 'PT'][['ICD-O Code', 'ICD-O string']],
    on='ICD-O Code',
    how='left'
)

print(counts_df.head(20))

   ICD-O Code  Count                                       ICD-O string
0      8000/6   1220                               Neoplasm, metastatic
1      8000/1    589    Neoplasm, uncertain whether benign or malignant
2      8000/3    242                                Neoplasm, malignant
3      8140/3     88                                Adenocarcinoma, NOS
4      8500/3     52                   Infiltrating duct carcinoma, NOS
5      8010/3     46                                     Carcinoma, NOS
6      8140/6     41                    Adenocarcinoma, metastatic, NOS
7      8720/3     33                            Malignant melanoma, NOS
8      8001/1     29  Tumor cells, uncertain whether benign or malig...
9      8001/3     25                             Tumor cells, malignant
10     8010/9     23                                     Carcinomatosis
11     8720/6     22                                                NaN
12     8010/6     21                         Carcinoma, metastat

Una vez tenemos el término que queremos buscar, en este caso "Adenocarcinoma, metastasic, NOS" (Código 8140/3), buscamos sus sinónimos para aumentar el texto a encontrar en las notas.

In [ ]:
def find_synonims(code):
    '''Función para encontrar los sinónimos de un código ICD-O.
        Input: código ICD-O (str).
        Output: lista de sinónimos (list of str).
    '''
    
    # Carga el mapa de códigos a términos
    ecie_umls_map = pd.read_csv(
        "../.data/mappings/ICD-O-3.1-NCIt_Morphology_Mapping.txt",
        sep = '\t',
        dtype=str,
        encoding='utf-8'
    )
    
    filtered = ecie_umls_map[(ecie_umls_map['ICD-O Code'] == code) & (ecie_umls_map['Term Type'] == 'SY')]
    
    # Si no tiene sinónimos, devuelve una lista vacía
    if filtered.empty:
        return []
    
    synonims = filtered['NCIt PT string (Preferred term)'].tolist()
    return synonims

# Ejemplo de uso
example_code = counts_df.iloc[6]['ICD-O Code']
syns = find_synonims(example_code)
print(f'Sinónimos para el código {example_code}: {syns}')

example_code = counts_df.iloc[0]['ICD-O Code']
syns = find_synonims(example_code)
print(f'Sinónimos para el código {example_code}: {syns}')

Sinónimos para el código 8140/6: []
Sinónimos para el código 8000/6: ['Secondary Neoplasm', 'Tumor Embolism', 'Metastatic Neoplasm', 'Secondary Neoplasm']


# Número de tokens por nota
Para asegurarme de que no me quedo sin espacio analizando las notas, voy a medir el número de tokens de cada una.

In [40]:
import os 
import glob
import numpy as np
from transformers import AutoTokenizer

import matplotlib.pyplot as plt
import numpy as np

def graficar_distribucion_tokens(token_counts, dataset_name:str, output_file:str="distribucion_tokens.png"):
    """ Genera un histograma con la distribución de tokens y marca los límites de contexto.
    """
    
    if len(token_counts) == 0:
        print("No hay datos para graficar.")
        return

    plt.figure(figsize=(12, 6))
    
    # 1. Crear el Histograma
    # 'bins=50' divide tus datos en 50 barras para ver el detalle
    plt.hist(token_counts, bins=50, color='#4C72B0', edgecolor='black', alpha=0.7, label='Frecuencia de Notas')
    
    # 2. Líneas de referencia estadística
    media = np.mean(token_counts)
    p95 = np.percentile(token_counts, 95)
    
    plt.axvline(media, color='red', linestyle='dashed', linewidth=1.5, label=f'Media ({int(media)})')
    plt.axvline(p95, color='purple', linestyle='dashed', linewidth=1.5, label=f'Percentil 95% ({int(p95)})')
    
    # 3. Líneas de Context Window (Las fronteras críticas)
    # Estos son los saltos típicos en configuración de LLMs
    context_limits = [2048, 4096, 8192, 16384]
    colors = ['orange', 'green', 'brown', 'gray']
    
    max_val = np.max(token_counts)
    
    for limit, color in zip(context_limits, colors):
        # Solo dibujamos la línea si está dentro del rango visual o cerca
        if limit < max_val * 1.2: 
            plt.axvline(limit, color=color, linestyle='dotted', linewidth=2, label=f'Límite {limit//1024}k')

    # 4. Estética
    plt.title(f'Distribución de Longitud de Tokens en {dataset_name}', fontsize=14)
    plt.xlabel('Número de Tokens', fontsize=12)
    plt.ylabel('Cantidad de Documentos', fontsize=12)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    # 5. Guardar
    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    print(f"📊 Gráfico guardado exitosamente en: {output_file}")
    plt.close()


def analizar_dataset(dataset_path:str, dataset_name:str, model_id:str):

    print(f"⬇️  Cargando tokenizador de {model_id}...")
    try:
        # Solo descarga el vocabulario (< 2MB usually), no el modelo entero
        tokenizer = AutoTokenizer.from_pretrained(model_id)
    except Exception as e:
        print(f"Error cargando tokenizador: {e}")
        return
    
    search_pattern = os.path.join(dataset_path, "**/*.txt")
    files = glob.glob(search_pattern, recursive=True)

    if not files:
        print(f"❌ No se encontraron archivos .txt en {dataset_path}")
        return
    
    print(f"📂 Se encontraron {len(files)} documentos. Analizando...")
    
    token_counts = []
    
    for file_path in files:
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()
                
            # Esta es la magia: codificar el texto a IDs de tokens y contar
            # Nota: No necesitamos attention_mask para esto
            tokens = tokenizer.encode(text, add_special_tokens=False)
            count = len(tokens)
            token_counts.append(count)
            
            # Opcional: Imprimir los archivos gigantes para inspeccionarlos luego
            if count > 6000: 
                print(f"⚠️  Archivo grande detectado ({count} tokens): {os.path.basename(file_path)}")
                
        except Exception as e:
            print(f"Error leyendo {file_path}: {e}")

    # --- ESTADÍSTICAS ---
    token_counts = np.array(token_counts)
    
    print("\n" + "="*40)
    print(f"📊 RESULTADOS DEL ANÁLISIS ({model_id})")
    print("="*40)
    print(f"Total documentos: {len(token_counts)}")
    print(f"🟢 Mínimo:          {np.min(token_counts)} tokens")
    print(f"🟡 Media (Promedio): {int(np.mean(token_counts))} tokens")
    print(f"🟠 Mediana:         {int(np.median(token_counts))} tokens")
    print(f"🔴 Máximo:          {np.max(token_counts)} tokens")
    print("-" * 40)
    print(f"Percentil 90:       {int(np.percentile(token_counts, 90))} tokens (El 90% de tus notas miden menos que esto)")
    print(f"Percentil 95:       {int(np.percentile(token_counts, 95))} tokens")
    print(f"Percentil 99:       {int(np.percentile(token_counts, 99))} tokens")
    print("="*40)

    # --- RECOMENDACIÓN AUTOMÁTICA ---
    max_tokens = np.max(token_counts)
    prompt_estimate = 500  # Espacio para tus instrucciones de sistema
    output_estimate = 2000 # Espacio para el JSON de salida
    
    needed_ctx = max_tokens + prompt_estimate + output_estimate
    
    print("\n💡 RECOMENDACIÓN DE INGENIERÍA:")
    print(f"Para cubrir el CASO PEOR (Nota más larga + Prompt + Salida):")
    print(f"Necesitas: {max_tokens} (Nota) + {prompt_estimate} (Sys) + {output_estimate} (Output) = {needed_ctx} tokens")
    
    standard_sizes = [2048, 4096, 8192, 16384, 32768]
    recommended = next((x for x in standard_sizes if x >= needed_ctx), "32k+")
    
    print(f"👉 Configura 'num_ctx' en Ollama a: {recommended}")

    graficar_distribucion_tokens(token_counts, dataset_name)



In [41]:
token_counts = analizar_dataset(
    dataset_path="/home/david/GitHub/MedText/.data/Cantemist/",
    model_id="Qwen/Qwen2.5-7B-Instruct",
    dataset_name="Cantemist All"
)

⬇️  Cargando tokenizador de Qwen/Qwen2.5-7B-Instruct...
📂 Se encontraron 8536 documentos. Analizando...
⚠️  Archivo grande detectado (9200 tokens): S1130-01082008001200008-1.txt
⚠️  Archivo grande detectado (6735 tokens): S1130-52742010000200007-1.txt
⚠️  Archivo grande detectado (7189 tokens): S1887-85712013000300006-1.txt
⚠️  Archivo grande detectado (6064 tokens): S0211-57352013000400004-1.txt
⚠️  Archivo grande detectado (7429 tokens): S1130-01082007000500007-1.txt

📊 RESULTADOS DEL ANÁLISIS (Qwen/Qwen2.5-7B-Instruct)
Total documentos: 8536
🟢 Mínimo:          30 tokens
🟡 Media (Promedio): 989 tokens
🟠 Mediana:         871 tokens
🔴 Máximo:          9200 tokens
----------------------------------------
Percentil 90:       1855 tokens (El 90% de tus notas miden menos que esto)
Percentil 95:       2158 tokens
Percentil 99:       2942 tokens

💡 RECOMENDACIÓN DE INGENIERÍA:
Para cubrir el CASO PEOR (Nota más larga + Prompt + Salida):
Necesitas: 9200 (Nota) + 500 (Sys) + 2000 (Output) = 11